In [9]:
#%pip install shapely
import os.path
import boto3
import pandas as pd
from datetime import datetime, timedelta
from datascience_implement_cache import DIC
from datascience_implement_cache.mongo import Mongo

In [10]:
s3 = boto3.client('s3')
dic = DIC('vr-devices')
dc_mongo = Mongo(dic)
db = dc_mongo.get_client("prod/bi/mongo")

In [4]:

buckets_test = list(db.buckets.find(
    {
        "type": 22,
        "created": {"$gte": datetime(2023, 1, 1)},        
        "files": {"$exists": True, "$not": {"$size": 0}},
        "conf.geoframe.map.center": {"$exists": True, "$not": {"$size": 0}},
        "conf.buckets._id": {"$exists": True, "$not": {"$size": 0}}
    },
    {
        "conf": 1,
        "orgId": 1,
        "files": 1,
        "name": 1,
        "created":1,
        "updated":1,
        "total_stat": "$stats.total"
        
    }
).limit(5))

In [ ]:
#Pull data into dataframe for following metrices:

#'bucket_ids','bucket_name','orgId','bucket_created','start_date','end_date', \
#'bucket_coordinates', 'matched_homes', 'matched_devices', 'unmatched_devices', 'extended_devices'

In [24]:
#values stored in separate lists
bucket_ids_main = []
bucket_name = []
org_id = []
bucket_created = []
start_date = []
end_date= []
bucket_coordinates = []
matched_homes = []
matched_devices = []
unmatched_devices = []
extended_devices = []

# Iterating over the buckets
for i,bucket in enumerate(buckets_test): 
    #print(i,bucket['_id'])
    bucket_ids_main.append(bucket['_id'])
    bucket_name.append(bucket['name'])
    org_id.append(bucket['orgId'])
    bucket_created.append(bucket['created'])
    start_date.append(bucket['conf']['geoframe']['timeframes'][0]['start'])
    end_date.append(bucket['conf']['geoframe']['timeframes'][0]['stop'])
    bucket_coordinates.append(bucket['conf']['geoframe']['map']['center']['coordinates'])
    matched_homes.append(bucket['conf']['buckets'][0]['stats']['total'])
    matched_devices.append(bucket['conf']['buckets'][1]['stats']['total'])
    unmatched_devices.append(bucket['conf']['buckets'][2]['stats']['total'])
    extended_devices.append(bucket['conf']['buckets'][3]['stats']['total'])
    
df_test_orgid = pd.DataFrame({'bucket_ids': bucket_ids_main, 'bucket_name':bucket_name, 'orgId': org_id, \
                              'bucket_created': bucket_created, 'start_date': start_date, 'end_date': end_date, \
                              'bucket_coordinates': bucket_coordinates, 'matched_homes': matched_homes, \
                              'matched_devices': matched_devices, 'unmatched_devices': unmatched_devices, \
                              'extended_devices': extended_devices
                   })

In [25]:
df_test_orgid

,bucket_ids,bucket_name,orgId,bucket_created,start_date,end_date,bucket_coordinates,matched_homes,matched_devices,unmatched_devices,extended_devices
0,DXKqXqJpx57rrwWzd,"GHF Competitors (Mattress Firm - Winchester, V...",58bdd7113314b55970a72e47,2023-01-03 14:52:43.826,2022-10-01 12:00:00.000,2022-12-29 03:00:00.000,"[-78.16641380000002, 39.1619316]",63,64,34,311
1,agfwsLsYcm2ix8PQG,Cuyahoga Community College (Cleveland),zkerk3AfSY4FpNrue,2023-01-03 15:45:53.194,2022-06-23 12:44:00.928,2022-12-23 19:44:00.928,"[-81.68358479999999, 41.4948929]",1714,1740,2374,7335
2,n2Q24yiBicY6eMe6n,River of Lights,sMrStG35oPZEpCCN3,2023-01-03 16:59:59.799,2022-12-03 23:00:00.000,2022-12-04 04:00:00.000,"[-98.4845574, 29.4332875]",72,73,101,360
3,JrMgHRfrhAJYmDoC4,Red Oak Bank Iowa Prospects - Town,JwmbatuumPxBtPmL3,2023-01-03 17:29:14.993,2022-12-01 05:00:00.000,2023-01-01 04:59:59.999,"[-95.2214477, 41.0170937]",2981,3568,7377,12226
4,Hc5N5546q2Qj8LL2x,Video @ Venue Replay @,fgcdnqnemFAuz3d8k,2023-01-03 19:40:43.453,2022-12-20 16:39:00.045,2022-12-20 22:39:00.045,"[-122.6175105, 45.6425651]",27,27,55,144


In [ ]:
df_test_orgid.to_csv('data_final.csv',index=False)

In [4]:
from vr_job_reversegeocode import VRJobLatLong

#Pass data in form of batch of chunks in to the reverseGeocode
chunk_size = 500  # Set your desired chunk size
df_chunks = pd.read_csv('data_final.csv', chunksize=chunk_size)

#latlong_df = pd.DataFrame(columns=['address', 'etHash', 'bucket_id'])  # Create an empty DataFrame with columns

for i, df_chunk in enumerate(df_chunks):
    # Split the 'coordinates' column into two columns: 'latitude' and 'longitude'
    df_chunk[['longitude', 'latitude']] = df_chunk['bucket_coordinates'].str.strip('[]').str.split(', ', expand=True)

    # Convert the columns to float (if needed)
    df_chunk['latitude'] = df_chunk['latitude'].astype(float)
    df_chunk['longitude'] = df_chunk['longitude'].astype(float)
    
    latlong_df = pd.DataFrame(columns=['address', 'etHash', 'bucket_id'])
    for index, row in df_chunk.iterrows():
        latitude = row['latitude'] 
        longitude = row['longitude']
        bucket_id = row['bucket_ids']
        vr_joblatlong1 = VRJobLatLong(latitude, longitude)
        vr_joblatlong1.run_job()
        address = vr_joblatlong1.address_match
        etHash = vr_joblatlong1.ethash_addr_match
        print(bucket_id, address)
        # Append a new row to the DataFrame using a dictionary
        latlong_df = latlong_df.append({'address': address, 'etHash': etHash,'bucket_id': bucket_id}, ignore_index=True)
        print(len(latlong_df))
    if i==0:
        latlong_df.to_csv('final_df_add.csv',index=False,header=True)
    else:
        latlong_df.to_csv('final_df_add.csv',index=False,header=False,mode='a')
# Print the resulting DataFrame
#print(latlong_df)

DXKqXqJpx57rrwWzd 1950 S PLEASANT VALLEY RD
WINCHESTER VA  22601
USA
1


/tmp/ipykernel_198110/2077245259.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latlong_df = latlong_df.append({'address': address, 'etHash': etHash,'bucket_id': bucket_id}, ignore_index=True)


agfwsLsYcm2ix8PQG 700 CARNEGIE AVE
CLEVELAND OH  44115
USA
2


/tmp/ipykernel_198110/2077245259.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latlong_df = latlong_df.append({'address': address, 'etHash': etHash,'bucket_id': bucket_id}, ignore_index=True)


n2Q24yiBicY6eMe6n 510 BROOKLYN AVE
SAN ANTONIO TX  78215
USA
3


...

EMdooHDzNfquiZ2Js 2626 E PECOS RD
CHANDLER AZ  85225
USA
488


/tmp/ipykernel_198110/2077245259.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latlong_df = latlong_df.append({'address': address, 'etHash': etHash,'bucket_id': bucket_id}, ignore_index=True)


Ko7NJm72zEbxzoA7s 1833 W SOUTHERN AVE
MESA AZ  85202
USA
489


/tmp/ipykernel_198110/2077245259.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  latlong_df = latlong_df.append({'address': address, 'etHash': etHash,'bucket_id': bucket_id}, ignore_index=True)


IndexError: list index out of range

In [2]:
df_final_noaddrs = pd.read_csv('data_final.csv') # Read data without etHash and address
df_final_addrs_ethash = pd.read_csv('final_df_add.csv') #data output from reversegeocode with address,etHash mapping



In [6]:
#Merge the data to mapp address and etHash
df_metric_mapping = pd.merge(df_final_noaddrs,df_final_addrs_ethash.rename(columns={'bucket_id': 'bucket_ids'}),on='bucket_ids',how='inner')

In [8]:
#write the final csv file
df_metric_mapping.to_csv('metric_df_vr.csv',index=False)

In [ ]:
#Mapping salesrep

In [6]:
#Read the metric file
df_metric_mapping = pd.read_csv('metric_df_vr.csv')

In [11]:

# List of _backet_id for mapping salesrep
id_list = list(df_metric_mapping['bucket_ids'])
buckets_test = list(db.buckets.find(
    {
        "_id": {"$in": id_list}, 
        "type": 22,
        "created": {"$gte": datetime(2023, 1, 1)},
        "files": {"$exists": True, "$not": {"$size": 0}},
        "conf.geoframe.map.center": {"$exists": True, "$not": {"$size": 0}},
        "conf.buckets._id": {"$exists": True, "$not": {"$size": 0}}
    },
    {
        "conf": 1,
        "orgId":1,
        "files": 1,
        "name": 1,
        "created": 1,
        "updated": 1,
        "total_stat": "$stats.total"
    }
))


In [12]:
len(buckets_test)

8155

In [ ]:
#Mapiing SalesRep to the file

In [13]:
# List of bucket_id values you want to match
orgId_list = list(df_metric_mapping['orgId'].unique())
buckets_reps = list(db.orgs.find(
    {
        "_id": {"$in": orgId_list} 
    },
    {
        "_id":1,
        "accountReps": 1
    }
))

NameError: name 'metric_df_vr' is not defined

In [51]:
len(buckets_reps)

470

In [73]:
#some buckets have >1 salesrep
buckets_reps[63]

{'_id': '58bdd7113314b55970a72e47',
 'accountReps': [{'name': 'Sean Howard',
   'email': 'showard@eltoro.com',
   'userId': '573092a2f1dbb830078d4bcd'},
  {'name': 'Meghan Darst',
   'email': 'meghan@eltoro.com',
   'userId': 'cuFGrFkBRckfjue9m'}]}

In [107]:
#capture salesrep and emails associated with backets and map to the metric dataframe

# Initialize an empty DataFrame
df_test_reps = pd.DataFrame()

# Iterate through the data and create columns dynamically
for i, item in enumerate(buckets_reps):
    _id = item.get('_id')
    account_reps = item.get('accountReps', [])

    # Create a dictionary for the row data
    row_data = {'orgId': _id}

    # Iterate through the account reps and create dynamic columns
    for j, rep in enumerate(account_reps, start=1):
        name_key = f'repName{j}'
        name_value = rep.get('name')
        row_data[name_key] = name_value
        
        email_key = f'repemail{j}'
        email_value = rep.get('email')
        row_data[email_key] = email_value
    # Append the row data to the DataFrame
    df_test_reps = df_test_reps.append(row_data, ignore_index=True)

# Print the resulting DataFrame
df_test_reps

/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWar

/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_test_reps = df_test_reps.append(row_data, ignore_index=True)
/tmp/ipykernel_98549/3138874623.py:22: FutureWar

,orgId,repName1,repemail1,repName2,repemail2
0,2Kcj4KTyMxQwMsrqZ,Maurice Williams,maurice@eltoro.com,NaN,NaN
1,2ML6sPYXc9v68HiXf,Crystal Norman,crystal@eltoro.com,NaN,NaN
2,2SjFXYMwsSLypAtwC,max@eltoro.com,max@eltoro.com,NaN,NaN
3,2Tv4ZWEZNptHX3zG8,Alyssa Rydell,alyssa.rydell@eltoro.com,Jeremy Sneed,jeremy.sneed@eltoro.com
4,2i9xHrXMRz9FpF5gn,Crystal Norman,crystal@eltoro.com,NaN,NaN
...,...,...,...,...,...
465,zfaoikzj4c64xHeqC,Marty Meyer,marty@eltoro.com,NaN,NaN
466,zj5w2CmzaogTBu4zH,Crystal Norman,crystal@eltoro.com,NaN,NaN
467,zkerk3AfSY4FpNrue,Sean Howard,showard@eltoro.com,Meghan Darst,meghan@eltoro.com
468,zuJ4vgf9dv3W3e9Cw,Meghan Waldrop,meghan.waldrop@eltoro.com,Brett Rebalsky,brett.rebalsky@eltoro.com


In [108]:
df_test_reps

,orgId,repName1,repemail1,repName2,repemail2
0,2Kcj4KTyMxQwMsrqZ,Maurice Williams,maurice@eltoro.com,NaN,NaN
1,2ML6sPYXc9v68HiXf,Crystal Norman,crystal@eltoro.com,NaN,NaN
2,2SjFXYMwsSLypAtwC,max@eltoro.com,max@eltoro.com,NaN,NaN
3,2Tv4ZWEZNptHX3zG8,Alyssa Rydell,alyssa.rydell@eltoro.com,Jeremy Sneed,jeremy.sneed@eltoro.com
4,2i9xHrXMRz9FpF5gn,Crystal Norman,crystal@eltoro.com,NaN,NaN
...,...,...,...,...,...
465,zfaoikzj4c64xHeqC,Marty Meyer,marty@eltoro.com,NaN,NaN
466,zj5w2CmzaogTBu4zH,Crystal Norman,crystal@eltoro.com,NaN,NaN
467,zkerk3AfSY4FpNrue,Sean Howard,showard@eltoro.com,Meghan Darst,meghan@eltoro.com
468,zuJ4vgf9dv3W3e9Cw,Meghan Waldrop,meghan.waldrop@eltoro.com,Brett Rebalsky,brett.rebalsky@eltoro.com


In [109]:
############QC#############
#check for buckets and count of salesrep associated
# Initialize a dictionary to store the count of names for each _id
count_names_dict = {}

# Iterate through the list of dictionaries
for item in buckets_reps:
    _id = item['_id']
    accountReps = item.get('accountReps', [])
    count_names = len(accountReps)
    count_names_dict[_id] = count_names

# Convert the count_names_dict to a DataFrame
df_count = pd.DataFrame(list(count_names_dict.items()), columns=['orgId', 'count_names'])

# Print the DataFrame
df_count

,orgId,count_names
0,2Kcj4KTyMxQwMsrqZ,1
1,2ML6sPYXc9v68HiXf,1
2,2SjFXYMwsSLypAtwC,1
3,2Tv4ZWEZNptHX3zG8,2
4,2i9xHrXMRz9FpF5gn,1
...,...,...
465,zfaoikzj4c64xHeqC,1
466,zj5w2CmzaogTBu4zH,1
467,zkerk3AfSY4FpNrue,2
468,zuJ4vgf9dv3W3e9Cw,2


In [110]:
############QC#############
#check for buckets having >1 salesrep
df_count[df_count['count_names']>1]

,orgId,count_names
3,2Tv4ZWEZNptHX3zG8,2
6,2oBQWKfWm5PgPmEuJ,2
11,3N8ZcSx24A76CbjsX,2
13,3axvuq2WHCPz7mFnJ,2
14,3co8wfeFESuGFLNBD,2
...,...,...
461,zGzzNFbGocJTvZkuS,2
462,zHRGCsCcARJLkq4vd,2
463,zKp5Kts7EQNAkC7TH,2
467,zkerk3AfSY4FpNrue,2


In [ ]:
#Join the metric tables with salesrep table

In [111]:
df_metric_ = pd.merge(df_metric_mapping_org,df_test_reps, on = 'orgId',how = 'left')

In [118]:
df_metric_

,bucket_ids,bucket_name,bucket_created,start_date,end_date,bucket_coordinates,matched_homes,matched_devices,unmatched_devices,extended_devices,address,etHash,orgId,repName1,repemail1,repName2,repemail2
0,DXKqXqJpx57rrwWzd,"GHF Competitors (Mattress Firm - Winchester, V...",2023-01-03 14:52:43.826,2022-10-01 12:00:00.000,2022-12-29 03:00:00.000,"[-78.16641380000002, 39.1619316]",63,64,34,311,1950 S PLEASANT VALLEY RD\nWINCHESTER VA 2260...,A2260144009044E0EB545930A111E8B9D0179472B6A7E7...,58bdd7113314b55970a72e47,Sean Howard,showard@eltoro.com,Meghan Darst,meghan@eltoro.com
1,agfwsLsYcm2ix8PQG,Cuyahoga Community College (Cleveland),2023-01-03 15:45:53.194,2022-06-23 12:44:00.928,2022-12-23 19:44:00.928,"[-81.68358479999999, 41.4948929]",1714,1740,2374,7335,700 CARNEGIE AVE\nCLEVELAND OH 44115\nUSA,A441152878DA4D361CA5CF7F1A3010F215243ED4934B62...,zkerk3AfSY4FpNrue,Sean Howard,showard@eltoro.com,Meghan Darst,meghan@eltoro.com
2,n2Q24yiBicY6eMe6n,River of Lights,2023-01-03 16:59:59.799,2022-12-03 23:00:00.000,2022-12-04 04:00:00.000,"[-98.4845574, 29.4332875]",72,73,101,360,510 BROOKLYN AVE\nSAN ANTONIO TX 78215\nUSA,A7821517004FB6E4C10F1DB139561DA6E8AEEDD38F68A1...,sMrStG35oPZEpCCN3,Daniel Kimball,dan@eltoro.com,Crystal Norman,crystal@eltoro.com
3,JrMgHRfrhAJYmDoC4,Red Oak Bank Iowa Prospects - Town,2023-01-03 17:29:14.993,2022-12-01 05:00:00.000,2023-01-01 04:59:59.999,"[-95.2214477, 41.0170937]",2981,3568,7377,12226,905 CHAUTAUQUA DR\nRED OAK IA 51566\nUSA,A515661201FE151C4D6804F2BFBE66E9512DE1722BC7D8...,JwmbatuumPxBtPmL3,Sean Howard,showard@eltoro.com,Meghan Darst,meghan@eltoro.com
4,Hc5N5546q2Qj8LL2x,Video @ Venue Replay @,2023-01-03 19:40:43.453,2022-12-20 16:39:00.045,2022-12-20 22:39:00.045,"[-122.6175105, 45.6425651]",27,27,55,144,2800 NE STAPLETON RD\nVANCOUVER WA 98661\nUSA,A986616823B924266415550A4091B2B9A2DB3FE89BF533...,fgcdnqnemFAuz3d8k,Meghan Waldrop,meghan.waldrop@eltoro.com,Brett Rebalsky,brett.rebalsky@eltoro.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8150,AwqaHuM54zaySewcc,Group G - YMS Epic Tournament 5/6-8/23 - Penns...,2023-09-27 18:31:33.939,2023-05-12 16:00:00.000,2023-05-14 22:30:00.000,"[-74.8231144, 40.21200169999999]",461,473,1596,2576,804 ROELOFFS RD\nMORRISVILLE PA 19067\nUSA,A190674601AC3BA50A9824675145821301837DF1454788...,7sJeuiTWBuCoMMHPq,Clay,clay@eltoro.com,NaN,NaN
8151,Lc8a7ji8PqWpgKhFN,Group G - YMS Epic Tournament 5/12-14/23 - Edg...,2023-09-27 18:34:18.714,2023-05-12 20:00:00.000,2023-09-26 22:00:00.000,"[-74.8367315, 40.22545849999999]",1287,1485,3723,6612,899 OXFORD VALLEY RD\nMORRISVILLE PA 19067\nUSA,A190671699E21760E8BCC077DEEED500A33B147EF942D3...,7sJeuiTWBuCoMMHPq,Clay,clay@eltoro.com,NaN,NaN
8152,GgJ9RsqBWPwsDEzEv,Group G - YMS Epic Tournament 5/12-14/23 - PHS...,2023-09-27 18:40:52.590,2023-05-12 21:00:00.000,2023-05-14 21:00:00.000,"[-74.8389248, 40.1743463]",156,165,451,823,608 S OLDS BLVD\nFAIRLESS HILLS PA 19030\nUSA,A190302499326656B088082013B2E8A1C77E8FF5E0EBEC...,7sJeuiTWBuCoMMHPq,Clay,clay@eltoro.com,NaN,NaN
8153,xRe8pXEkvuKTuA5Jw,new_addresses_non model,2023-09-27 18:48:25.868,2023-08-01 04:00:00.000,2023-09-01 03:59:59.999,"[-74.2374043, 40.7241484]",1853,1867,4200,8696,660 STUYVESANT AVE\nIRVINGTON NJ 07111\nUSA,A07111182401F56D4606E319B36854FDFEADFA77171364...,qkEKxRL9h3NCjKp9k,Patrick Carleton,pc@eltoro.com,NaN,NaN


In [122]:
#create URL for the buckets
df_metric_['URL']= "https://portal.eltoro.com/bucket/" + df_metric_['bucket_ids'].astype(str)


In [125]:
#check for null values in the data
df_metric_.isna().sum()

bucket_ids               0
bucket_name              0
bucket_created           0
start_date               0
end_date                 0
bucket_coordinates       0
matched_homes            0
matched_devices          0
unmatched_devices        0
extended_devices         0
address                  0
etHash                  36
orgId                    0
repName1                 0
repemail1                0
repName2              2875
repemail2             2875
URL                      0
dtype: int64

In [128]:
#create seperate dataframe each for salesrep
# Get unique values in the 'email1' column
unique_emails = df_metric_['repemail1'].unique()

# Iterate over unique email values and create separate CSV files
for email in unique_emails:
    # Create a subset DataFrame for the current email value
    subset_df = df_metric_[df_metric_['repemail1'] == email]
    
    # Define the output CSV file name
    output_file = f'email_list/{email}.csv'
    
    # Write the subset DataFrame to a CSV file
    subset_df.to_csv(output_file, index=False)

print("CSV files created successfully.")

CSV files created successfully.
